# BTM

## Imports

In [15]:
import bitermplus as btm
import numpy as np
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
import gensim
from tqdm import tqdm

## Load data

In [2]:
def import_data():
    my_df = pd.read_csv('../data/to_be_clustered.csv.gz', compression="gzip")
    my_df = pd.DataFrame(my_df)
    return my_df

def clean_data(my_df, pays):
    my_df['clean'].apply(lambda x: str(x))
    #in_df = my_df[my_df['whcs'] == "France"]
    in_df = my_df[my_df['whcs'] == pays]
    in_df = in_df['clean'].dropna()
    texts = in_df.str.strip().tolist()
    
    return texts

## Preprocessing

In [ ]:
def btm_preprocessing(texts):

    # Obtaining terms frequency in a sparse matrix and corpus vocabulary
    X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
    tf = np.array(X.sum(axis=0)).ravel()
    # Vectorizing documents
    docs_vec = btm.get_vectorized_docs(texts, vocabulary)
    docs_lens = list(map(len, docs_vec))
    # Generating biterms
    biterms = btm.get_biterms(docs_vec)
    
    return X, vocabulary, biterms, docs_vec

## Create model

In [16]:
def create_model(X, vocabulary, seed, alpha, beta, biterms, docs_vec):
    
    model = btm.BTM(
    X, vocabulary, seed=seed, T=10, M=20, alpha=alpha, beta=beta)
    model.fit(biterms, iterations=20)
    p_zd = model.transform(docs_vec)
    
    return model

## Metrics

In [ ]:
def compute_coherence(my_df, model):
    # cast tweets to numpy array
    docs = my_df.clean.apply(lambda x: str(x).split()).to_numpy()
    # create dictionary of all words in all documents
    dictionary = gensim.corpora.Dictionary(docs)
    # create BOW dictionary
    bow_corpus = [dictionary.doc2bow(doc) for doc in docs]
    top_words = btm.get_top_topic_words(model, words_num=10)
    cm_gsdmm = CoherenceModel(topics=top_words.T.to_numpy(), 
                          dictionary=dictionary, 
                          corpus=bow_corpus,
                          texts=docs, 
                          coherence='c_v')
    coherence = cm_gsdmm.get_coherence()  # get coherence value
    
    return coherence

## Find parameters

In [42]:
def grid_search(my_df, low_seed, high_seed, low_alpha, high_alpha, low_beta, high_beta, X, vocabulary, biterms, docs_vec):
    best=0
    best_param = {}
    for seed in range(low_seed, high_seed, 100):
        for alpha in np.arange(low_alpha, high_alpha, 0.01):
            for beta in np.arange(low_beta, high_beta, 0.01):
                model = create_model(X, vocabulary, seed, alpha, beta, biterms, docs_vec)
                coherence = compute_coherence(my_df, model)
                if (coherence > best) :
                    best = coherence
                    best_param['seed'] = seed
                    best_param['alpha'] = alpha
                    best_param['beta'] = beta
                    
    return best, best_param
        

In [43]:
def simulate(low_seed, high_seed, low_alpha, high_alpha, low_beta, high_beta):
    my_df = import_data()
    texts = clean_data(my_df, "France")
    X, vocabulary, biterms, docs_vec = btm_preprocessing(texts)
    best, best_param = grid_search(my_df, low_seed, high_seed, low_alpha, high_alpha, low_beta, high_beta, X, vocabulary, biterms, docs_vec)
    print('best', best)
    print('best params', best_param)
    return best, best_param

In [44]:
best, best_param = simulate(1, 12000, 0.0000001, 5, 0.000001, 5)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 75292.68it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 75483.88it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 82804.11it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 99577.24it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 81411.18it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 101156.94it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 42881.67it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 93815.98it/s]


 30%|██████████████████████████████████████████████████▍                                                                                                                     | 6/20 [00:00<00:00, 28.07it/s]


KeyboardInterrupt: 

In [41]:
best

NameError: name 'best' is not defined

## Reproduce with best parameters

In [ ]:
def best_model():
    my_df = import_data()
    texts = clean_data(my_df, "France")
    X, vocabulary, biterms, docs_vec = btm_preprocessing(texts)
    model = create_model(X, vocabulary, 1000, 0.0000001, 1, biterms, docs_vec)
    coherence = compute_coherence(my_df, model)
    print(coherence)